In [47]:
# ! pip install torch transformers peft pandas  tqdm

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim

from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, LoraConfig, get_peft_model
from tqdm import tqdm

from torch.nn.utils.rnn import pad_sequence
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import DataLoader, Dataset

import os
import pandas as pd

MY_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(MY_DEVICE)

cpu


In [49]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
from huggingface_hub import login
login('')
!huggingface-cli whoami

Mathwizard1


In [51]:
MY_ADAPTERS = {
    #"Politeness": ({0: "Not-polite", 1: "Polite"}, {"Not-polite": 0, "Polite": 1}), 
    #"Toxicity": ({0: "Non-Toxic", 1: "Toxic"}, {"Non-Toxic": 0, "Toxic": 1}), 
    #"Fluency": ({0: "Not-Fluent", 1: "Fluent"}, {"Not-Fluent": 0, "Fluent": 1}), 
    #"Convincing": ({0: "Non-convincing", 1: "Convincing"}, {"Non-convincing": 0, "Convincing": 1}),
    #"Factual": ({0: "fake-news", 1: "real-news"}, {"fake-news": 0, "real-news": 1})
}


D_TYPE = torch.bfloat16
SAVE_DIR = "MultiDiscriModel"

# Selected Hyperparameters
MAX_SEQUENCE_LENGTH = 512
TRUNCATION = True
PADDING = "max_length"
RETURN_TENSORS = "pt"
PADDING_SIDE= "right"

model_id = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer_id = model_id

####################################
if(not os.path.exists(SAVE_DIR)):
    os.mkdir(SAVE_DIR)

In [52]:
generator = AutoModelForCausalLM.from_pretrained(
        model_id, 
        torch_dtype= D_TYPE,
        max_length = MAX_SEQUENCE_LENGTH,
        )

generator.config.use_cache= True

In [53]:
print(f"Memory footprint: {generator.get_memory_footprint() / 1e6:.2f} MB")

Memory footprint: 2471.63 MB


In [54]:
tokenizer = AutoTokenizer.from_pretrained(
        tokenizer_id,
        padding_side= "right")

In [55]:
# Access special tokens and their IDs
print(f"BOS token: {tokenizer.bos_token}, ID: {tokenizer.bos_token_id}")
print(f"EOS token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")
print(f"CLS token: {tokenizer.cls_token}, ID: {tokenizer.cls_token_id}")
print(f"SEP token: {tokenizer.sep_token}, ID: {tokenizer.sep_token_id}")
print(f"PAD token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(f"UNK token: {tokenizer.unk_token}, ID: {tokenizer.unk_token_id}")
print(f"MASK token: {tokenizer.mask_token}, ID: {tokenizer.mask_token_id}")

# Set the end of string tokens
if(tokenizer.pad_token is None):
    tokenizer.pad_token_id = tokenizer.eos_token_id
    tokenizer.pad_token = tokenizer.eos_token
    print(f"PAD token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")

BOS token: <|begin_of_text|>, ID: 128000
EOS token: <|eot_id|>, ID: 128009
CLS token: None, ID: None
SEP token: None, ID: None
PAD token: None, ID: None
UNK token: None, ID: None
MASK token: None, ID: None
PAD token: <|eot_id|>, ID: 128009


In [56]:
def set_padding(model):
    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.pad_token = tokenizer.pad_token

set_padding(generator)
print(generator.config.pad_token_id)

128009


In [57]:
# === Apply LoRA ===
lora_config = LoraConfig(
    r= 64,                                    # Rank
    lora_alpha= 128,                          # Scaling factor
    target_modules=["q_proj", "v_proj"],    # Typical for LLaMA models
    lora_dropout= 0.05,
    bias="none",
    task_type="CAUSAL_LM",        
)

generator = get_peft_model(generator, lora_config)
generator.print_trainable_parameters()

trainable params: 6,815,744 || all params: 1,242,630,144 || trainable%: 0.5485


In [58]:
discriminators = []
for adapter in MY_ADAPTERS:
    ID2LABEL, LABEL2ID = MY_ADAPTERS[adapter]

    discriminator = AutoModelForSequenceClassification.from_pretrained(
        model_id, 
        torch_dtype= D_TYPE,
        max_length = MAX_SEQUENCE_LENGTH,

        # Model label map
        num_labels= 2,
        id2label= ID2LABEL,
        label2id= LABEL2ID,
        )
    
    #discriminator = get_peft_model()
    discriminators.append((discriminator, adapter))

In [59]:
Extra_prompts = {
    "Politeness": "Check for words like Thank you, Please, Welcome, very much, and other words of gratitude, courtesy and politeness.", 
    "Toxicity": "Check for presence of any Toxic, Hateful, Targeted, or any form of speech or profanity aimed to hurt sentiments.", 
    "Fluency": "Check for bad grammar, spelling mistakes, unintelligible or incomprehensible words and sentences. Keep check for brevity, clarity and unity between the statements.", 
    "Factual": "Give more attention to named entity, dates and numbers, general facts, consistent statements.",
    "Convincing": "Check if the statement seems convincing to the user in terms of language and message to inform the user."
}

def prompt_template(sentence: str, adapter: str):
    ID2LABEL, _ = MY_ADAPTERS[adapter]

    return f'''  
    Classification: 0 for {ID2LABEL[0]} and 1 for {ID2LABEL[1]}.
    {Extra_prompts[adapter]}
    Sentence => {sentence}
    '''

In [60]:
def input_prompt(sentence: str):
    return f'''
Craft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.
Explain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.
Conclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.
    fake news: {sentence}
    counter narrative:
    '''

In [61]:
####################################
# Training Parameters
LEARNING_RATE = 2e-4
WEIGHT_DECAY = 0.001

BATCH_SIZE = 8  # Increased for stability
NUMBER_OF_EPOCHS = 20

GRAD_CLIP = 1.0  # Added for stability

CLEAR_CACHE = False
####################################
TRAIN_DATA_PATH = 'final_dataset.csv'
RAND_SHUFFLE = False
NUM_ROWS = 2

In [62]:
# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe_addrs, max_length=128, num_rows= None):
        dataframe = pd.read_csv(dataframe_addrs)
        if(num_rows):
            dataframe = pd.read_csv(dataframe_addrs, nrows= num_rows)
            
        print(dataframe.shape)
        self.length = len(dataframe)

        self.prompts = dataframe['fake_news']
        self.targets = dataframe['counter_narrative_with_facts']

        self.max_length = max_length

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        prompt = input_prompt(self.prompts[idx])
        target = self.targets[idx]

        prompt_ids = tokenizer.encode(prompt, add_special_tokens=True)
        target_ids = tokenizer.encode(target, add_special_tokens=False)

        input_ids = prompt_ids + target_ids
        attention_mask = [1] * len(input_ids)

        labels = [-100] * len(prompt_ids) + target_ids

        padding_length = self.max_length - len(input_ids)
        if padding_length > 0:
            input_ids += [tokenizer.pad_token_id] * padding_length
            attention_mask += [0] * padding_length
            labels += [-100] * padding_length
        else:
            input_ids = input_ids[:self.max_length]
            attention_mask = attention_mask[:self.max_length]
            labels = labels[:self.max_length]

        return {
            "prompts": prompt,
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "labels": torch.tensor(labels),
        }

In [63]:
####################################
sentence_loader = DataLoader(
            TextDataset(TRAIN_DATA_PATH, max_length= MAX_SEQUENCE_LENGTH, num_rows= NUM_ROWS), 
            batch_size= BATCH_SIZE, shuffle= RAND_SHUFFLE,

            # Access data from CPU faster
            pin_memory= True,
            )


for batch in sentence_loader:
    print(batch['input_ids'].shape)
    print(batch['attention_mask'].shape)
    print(batch['labels'].shape)
    break

(2, 2)
torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 512])


In [64]:
def check_decodeprompt():
    for batch in sentence_loader:
        print(batch['input_ids'].shape)
        # print(batch['attention_mask'])

        for sentence in tokenizer.batch_decode(batch['input_ids']):
            print(str(sentence))
        break
check_decodeprompt()

torch.Size([2, 512])
<|begin_of_text|>
Craft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.
Explain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.
Conclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.
    fake news: [
     'A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump's hotel room five years ago, claiming she was hired by Donal

In [65]:
####################################
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(generator.parameters(), lr= LEARNING_RATE, weight_decay= WEIGHT_DECAY)

In [66]:
# Training function
def train_model(model, tokenizer, dataloader, num_epochs, save_path):
    torch.cuda.empty_cache()
    
    model.train()
    best_loss = float('inf')

    for epoch in range(num_epochs):
        total_loss = 0.0
        tqdm_loop = tqdm(dataloader, total= len(dataloader))
        
        for batch in tqdm_loop:
            input_ids = batch['input_ids'].to(MY_DEVICE)
            attention_mask = batch['attention_mask'].to(MY_DEVICE)
            labels = batch['labels'].to(MY_DEVICE)

            outputs = model(input_ids= input_ids, attention_mask= attention_mask, labels= labels)

            loss = outputs.loss

            #### Discriminator section ####
            with torch.no_grad():
                batch_prompts = batch["prompts"]  # get original prompts list

                # Tokenize prompts only
                prompt_encodings = tokenizer(batch_prompts, return_tensors= RETURN_TENSORS, max_length= MAX_SEQUENCE_LENGTH,
                                            padding= PADDING, truncation= TRUNCATION).to(MY_DEVICE)

                # Generate paraphrased output
                generated_ids = model.generate(
                    input_ids=prompt_encodings["input_ids"],
                    attention_mask=prompt_encodings["attention_mask"],
                    max_new_tokens= MAX_SEQUENCE_LENGTH,
                )

                generated_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
                print(generated_texts)

            break
            #### #### #### ####

            combined_loss = loss # + Disc_loss
            combined_loss.backward()

            # Gradient clipping
            clip_grad_norm_(generator.parameters(), max_norm= GRAD_CLIP)

            optimizer.step()
            optimizer.zero_grad()


            # Print loss at the end of epoch
            tqdm_loop.set_description(f"Epoch [{epoch+1}/{num_epochs}")
            tqdm_loop.set_postfix(loss= combined_loss.item())
            
            total_loss += combined_loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

        # Save best model
        if avg_loss < best_loss:
            best_loss = avg_loss
            model.save_pretrained(save_path)
            print(f"model saved after epoch {epoch+1}")

    print("Training Done!")

train_model(generator, tokenizer, sentence_loader, NUMBER_OF_EPOCHS, SAVE_DIR)

  0%|          | 0/1 [06:17<?, ?it/s]

["\nCraft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.\nExplain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.\nConclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.\n    fake news: [\n     'A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump's hotel room five years ago, claiming she was hired by Donald Trump along with two of her f

model saved after epoch 1


  0%|          | 0/1 [05:46<?, ?it/s]


["\nCraft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.\nExplain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.\nConclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.\n    fake news: [\n     'A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump's hotel room five years ago, claiming she was hired by Donald Trump along with two of her f

...

["\nCraft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.\nExplain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.\nConclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.\n    fake news: [\n     'A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump's hotel room five years ago, claiming she was hired by Donald Trump along with two of her f

  0%|          | 0/1 [06:36<?, ?it/s]


["\nCraft a single, polite paragraph addressing misinformation neutrally. Use verifiable facts to respectfully correct false claims and highlight inconsistencies.\nExplain the harm of misinformation and the value of evidence-based thinking. Maintain a kind, non-toxic tone focused solely on facts.\nConclude by encouraging critical thinking and reliance on verified information. Refer to the fake news to stay relevant to the topic.\n    fake news: [\n     'A Russian sex worker said Donald Trump was one of her clients and that he has a small penis. A 22-year sex worker named Ivana Kamensky, claims she was one of the prostitutes who had sexual intercourse with Donald Trump in Moscow in 2011, adding that the American President had the smallest penis she had ever seen. In an interview with the Moscow Daily Herald, the young woman described in great details the night that she allegedly spent in Mr Trump's hotel room five years ago, claiming she was hired by Donald Trump along with two of her f

  0%|          | 0/1 [03:46<?, ?it/s]


KeyboardInterrupt: 